In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from PIL import Image, ImageDraw

In [12]:

def to_img(doodle_data, img_size=(256, 256), padding=0.2):
    image = Image.new("L", img_size, 255)
    draw = ImageDraw.Draw(image)

    x_min = min(min(stroke[0]) for stroke in doodle_data)
    x_max = max(max(stroke[0]) for stroke in doodle_data)
    y_min = min(min(stroke[1]) for stroke in doodle_data)
    y_max = max(max(stroke[1]) for stroke in doodle_data)

    doodle_width = x_max - x_min
    doodle_height = y_max - y_min

    available_width = img_size[0] * (1 - padding)
    available_height = img_size[1] * (1 - padding)

    aspect_ratio = doodle_width / doodle_height if doodle_height != 0 else 1
    img_aspect_ratio = available_width / available_height

    if aspect_ratio > img_aspect_ratio:
        scale = available_width / doodle_width
    else:
        scale = available_height / doodle_height

    x_offset = (img_size[0] - doodle_width * scale) / 2
    y_offset = (img_size[1] - doodle_height * scale) / 2

    for stroke in doodle_data:
        x, y, _ = stroke

        scaled_x = [x_offset + (v - x_min) * scale for v in x]
        scaled_y = [y_offset + (v - y_min) * scale for v in y]

        draw.line(list(zip(scaled_x, scaled_y)), fill=0, width=2)

    image_gray = np.array(image)

    return image_gray


### Preprocess

In [23]:
def create_processed_img(read_path, save_path):
    lim = 10_000
    jsons = []
    with open(read_path) as file:
        for i in range(lim):
            jsons.append(json.loads(file.readline()))
    
    drawings = [item['drawing'] for item in jsons]
    drawings = [to_img(item) for item in drawings]
    drawings = [np.reshape(item, (1, 256 * 256)) for item in drawings]
    drawings = np.array(drawings)
    np.save(save_path, drawings)


In [24]:
create_processed_img('Quickdraw/full_raw_airplane.ndjson', 'Processed/airplane.npy')

In [25]:
create_processed_img('Quickdraw/full_raw_bicycle.ndjson', 'Processed/bicycle.npy')

In [26]:
create_processed_img('Quickdraw/full_raw_ear.ndjson', 'Processed/ear.npy')

In [27]:
create_processed_img('Quickdraw/full_raw_rabbit.ndjson', 'Processed/rabbit.npy')